In [16]:
from biopandas.mol2 import PandasMol2
import rdkit
import os
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import math
from scipy.ndimage.filters import convolve
import numpy as np

In [9]:
resolution_size_1 = 80
resolution_size_2 = 80
resolution_size_3 = 80
channels = 4
kernel_size = 16

In [13]:
def compute_offset_matrix_wave(kernel_size):
    x = range(-kernel_size/2,kernel_size/2)
    y = range(-kernel_size/2,kernel_size/2)
    z = range(-kernel_size/2,kernel_size/2)
    matrix = np.zeros((kernel_size,kernel_size,kernel_size))
    sigma = 4.0
    w = 1/sigma
    clip_distance = 4*sigma
    for i in x:
        for j in y:
            for k in z:
                if np.linalg.norm([i,j,k]) < clip_distance:
                    exp_part = (i**2)+(j**2)+(k**2)
                    denom_exp_part = 2*(sigma**2)
                    matrix[i+kernel_size/2][j+kernel_size/2][k+kernel_size/2] = math.exp(-(exp_part/denom_exp_part))* math.cos(2*math.pi*w*math.sqrt(exp_part))

    return matrix

In [14]:
def convert_to_indeces(dimension_x,dimension_y,dimension_z,molecule):
    scale_x = MinMaxScaler((8,dimension_x-8))
    scale_y = MinMaxScaler((8,dimension_y-8))
    scale_z = MinMaxScaler((8,dimension_z-8))
    scaled_molecule = pd.DataFrame()
    scaled_molecule['x'] = scale_x.fit_transform(molecule[['x']]).flatten()
    scaled_molecule['y'] = scale_y.fit_transform(molecule[['y']]).flatten()
    scaled_molecule['z'] = scale_z.fit_transform(molecule[['z']]).flatten()
    final_molecule = pd.DataFrame()
    final_molecule['x'] = scaled_molecule['x'].apply(lambda x: round(x))
    final_molecule['y'] = scaled_molecule['y'].apply(lambda x: round(x))
    final_molecule['z'] = scaled_molecule['z'].apply(lambda x: round(x))
    errors = pd.DataFrame()
    errors['x'] = np.abs(final_molecule['x'] - scaled_molecule['x'])/dimension_x
    errors['y'] = np.abs(final_molecule['y'] - scaled_molecule['y'])/dimension_y
    errors['z'] = np.abs(final_molecule['z'] - scaled_molecule['z'])/dimension_z
    atoms = pmol.df.atom_name.values
    return final_molecule,errors,atoms

In [17]:
kernel_matrix = compute_offset_matrix_wave(kernel_size)

In [22]:
pmol = PandasMol2().read_mol2('/Users/stevenagajanian/Documents/School/Grad_School/Thesis/gan/split_kinases/ABL/1.mol2')

AssertionError: 9 columns passed, passed data had 6 columns

In [20]:
encoding_mapper = {'C':0,'O':1,'N':2,'H':3}
decoding_mapper = {0:'C',1:'O',2:'N',3:'H'}
for j in os.listdir('/Users/stevenagajanian/Documents/School/Grad_School/Thesis/gan/split_kinases')[1:2]:
    for k in os.listdir('/Users/stevenagajanian/Documents/School/Grad_School/Thesis/gan/split_kinases/'+j):
        print '/Users/stevenagajanian/Documents/School/Grad_School/Thesis/gan/split_kinases/'+j+'/'+k
        pmol = PandasMol2().read_mol2('/Users/stevenagajanian/Documents/School/Grad_School/Thesis/gan/split_kinases/'+j+'/'+k)
        final_molecule,error,atoms = convert_to_indeces(64,64,64,pmol.df[['x','y','z']])
        attempted_molecule_ohe = np.zeros((resolution_size_1,resolution_size_2,resolution_size_3,channels)) 
        for i in range(final_molecule.shape[0]):
            attempted_molecule_ohe[int(final_molecule.iloc[i]['x'])][int(final_molecule.iloc[i]['y'])][int(final_molecule.iloc[i]['z'])][encoding_mapper[atoms[i][0]]] = 1
        transformed_molecule = np.zeros((resolution_size_1,resolution_size_2,resolution_size_3,channels))
        transformed_molecule[:,:,:,0]= convolve(attempted_molecule[0], kernel_matrix,mode='constant')
        transformed_molecule[:,:,:,1]= convolve(attempted_molecule[1], kernel_matrix,mode='constant')
        transformed_molecule[:,:,:,2]= convolve(attempted_molecule[2], kernel_matrix,mode='constant')
        transformed_molecule[:,:,:,3]= convolve(attempted_molecule[3], kernel_matrix,mode='constant')

/Users/stevenagajanian/Documents/School/Grad_School/Thesis/gan/split_kinases/ABL/0.mol2


AssertionError: 9 columns passed, passed data had 6 columns

In [3]:
pmol = PandasMol2().read_mol2('/Users/stevenagajanian/Documents/School/Grad_School/Thesis/gan/kinases/ZAP70.mol2')

In [4]:
print('Molecule ID: %s' % pmol.code)
print('\nRaw MOL2 file contents:\n\n%s\n...' % pmol.mol2_text)

Molecule ID: ZINC000026986400

Raw MOL2 file contents:

@<TRIPOS>MOLECULE
ZINC000026986400
64    67    1     0     0
SMALL
USER_CHARGES
@<TRIPOS>ATOM
1      C1     -1.4779    1.2205   -0.6895 C.3      1 ZINC000026986400   -0.1700
2      C2     -1.2415   -0.2875   -0.5843 C.3      1 ZINC000026986400   -0.1500
3      C3     -0.5332   -0.7820   -1.8471 C.3      1 ZINC000026986400    0.0300
4      H4      0.3810   -0.2082   -1.9991 H        1 ZINC000026986400    0.1500
5      C5     -0.1852   -2.2638   -1.6897 C.3      1 ZINC000026986400    0.0400
6      N6      0.4778   -2.7409   -2.9115 N.3      1 ZINC000026986400   -0.4800
7      C7      0.8570   -4.0697   -2.7851 C.ar     1 ZINC000026986400    0.3000
8      C8      1.5081   -4.7020   -3.8413 C.ar     1 ZINC000026986400   -0.1800
9      C9      1.8821   -6.0239   -3.7080 C.ar     1 ZINC000026986400    0.0000
10     C10     1.5895   -6.6809   -2.5065 C.ar     1 ZINC000026986400   -0.1100
11     C11     1.9708   -8.0998   -2.3140 C.ar    

In [5]:
pmol.df

,atom_id,atom_name,x,y,z,atom_type,subst_id,subst_name,charge
0,1,C1,-1.4779,1.2205,-0.6895,C.3,1,ZINC000026986400,-0.17
1,2,C2,-1.2415,-0.2875,-0.5843,C.3,1,ZINC000026986400,-0.15
2,3,C3,-0.5332,-0.7820,-1.8471,C.3,1,ZINC000026986400,0.03
3,4,H4,0.3810,-0.2082,-1.9991,H,1,ZINC000026986400,0.15
4,5,C5,-0.1852,-2.2638,-1.6897,C.3,1,ZINC000026986400,0.04
5,6,N6,0.4778,-2.7409,-2.9115,N.3,1,ZINC000026986400,-0.48
6,7,C7,0.8570,-4.0697,-2.7851,C.ar,1,ZINC000026986400,0.30
7,8,C8,1.5081,-4.7020,-3.8413,C.ar,1,ZINC000026986400,-0.18
8,9,C9,1.8821,-6.0239,-3.7080,C.ar,1,ZINC000026986400,0.00
9,10,C10,1.5895,-6.6809,-2.5065,C.ar,1,ZINC000026986400,-0.11
